<h1>Análise de procedimentos hospitalares do SUS</h1>

<h2>Módulo 1</h2>

# Aula 1

## Primeiros passos

Neste documento, será feita uma análise do custo total de procedimentos hospitalares, separados por Estado e medidos mensalmente.
O primeiro passo é importar as bibliotecas necessárias.

In [1]:
import pandas as pd # Manipulação de tratamento de dados
import matplotlib.pyplot as plt # Plotagem de gráficos
import matplotlib.ticker as ticker # Formatação de dados de gráficos
import numpy as np # Biblioteca Numpy

A segunda etapa é realizar a leitura dos dados. Neste caso, usamos o dataset 'A192613189_28_143_208.csv', disponível no <a href='http://tabnet.datasus.gov.br/cgi/tabcgi.exe?sih/cnv/qiuf.def'>Datasus</a>.

In [2]:
dados = pd.read_csv('A192613189_28_143_208.csv', encoding='ISO-8859-1', skiprows=3, sep=';', skipfooter=12, engine='python', thousands='.', decimal=',')
dados

FileNotFoundError: [Errno 2] No such file or directory: 'A192613189_28_143_208.csv'

Como é possível verificar acima, os números estão sendo apresentados em notação científica. Para melhorar a legibilidade dos mesmos, vamos formatar globalmente o Pandas para formatar números de ponto flutuante com precisão de duas casas decimais.

In [ ]:
pd.options.display.float_format = "{:.2f}".format

E agora nos certificamos que a formação está correta!

In [ ]:
dados

Apenas para testes, podemos verificar a média total do dataset.

In [ ]:
dados.mean()

## Iniciando com gráficos

A partir daqui vamos começar a plotar alguns gráficos. Este primeiro mostrará os dados referentes a Agosto de 2008, somente informando os eixos X e Y.

In [ ]:
ax = dados.plot(x='Unidade da Federação', y='2008/Ago', kind='bar', figsize=(15,8))
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:,.2f}"))
plt.title('Valor por unidade da federação')
plt.show()

É possível melhorar a exibição deste gráfico, começando por ordenar os valores e apresenta-los na horizontal, exibindo valores por milhão de reais.

In [ ]:
dados_ordenados = dados.sort_values(by='2008/Ago', ascending=False)
ax = dados_ordenados.plot(x='Unidade da Federação', y='2008/Ago', kind='barh', figsize=(15,8))
ax.invert_yaxis()
ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.2f}'))
plt.title('Valor por unidade da federação')
plt.show()

## Desafios

### 1 - Escolher um título mais descritivo

Para cumprir o primeiro desafio, segue o mesmo gráfico, porém com um título mais descritivo e o eixo X (valores) divido por um milhão, para facilitar a legibilidade.

In [ ]:
dados_ordenados['2008/Ago'] /= 1000000
ax = dados_ordenados.plot(x='Unidade da Federação', y='2008/Ago', kind='barh', figsize=(15,8))
ax.invert_yaxis()
ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.2f}'))
plt.title(label = 'Valor total gasto pelo SUS em procedimentos hospitalares por Estado (em milhão de reais)', fontsize=20)
plt.show()

### 2 - Mesma análise da aula, porém com o mês mais recente

Para cumprir o segundo desafio, precisaremos saber qual o mês mais recente disponível no dataset. Como os meses estão distribuídos em colunas, podemos realizar uma consulta no dataset para retornar, por exemplo, as últimas três colunas disponíveis.

In [ ]:
dados[dados.columns[-3:]].head()

Dado o resultado acima, verificamos que o mês mais recente disponível para análise é Março de 2021. Então vamos replicar o gráfico feito anteriormente, atualizando o mês e ano. Existe uma linha na coluna referente ao mês de março que contém um hífen no lugar de um valor numérico. Para evitar erros durante a execução do código, vamos substituir esta ocorrência por um zero e, em seguida, plotar o gráfico.

In [ ]:
#dados['2021/Mar'] = dados['2021/Mar'].str.replace(r'-', '0').astype(float)
dados_mais_recentes = dados.sort_values(by='2021/Mar', ascending=False)
dados_mais_recentes['2021/Mar'] /= 1000000
ax = dados_mais_recentes.plot(x='Unidade da Federação', y='2021/Mar', kind='barh', figsize=(15,8))
ax.invert_yaxis()
ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.2f}'))
plt.title(label = 'Valor total gasto pelo SUS em procedimentos hospitalares por Estado (em milhão de reais)', fontsize=20)
plt.show()

# Aula 2

## Importando dataset de atendimentos

A partir deste ponto faremos o mesmo procedimento de import de dados realizado na aula passada, porém com um dataset de atendimentos do SUS. O dataset foi obtido por meio <a href='http://tabnet.datasus.gov.br/cgi/tabcgi.exe?sih/cnv/qiuf.def'>deste link</a>.

In [ ]:
dados_atendimentos = pd.read_csv('A224855189_28_143_208.csv', encoding='ISO-8859-1', skiprows=3, sep=';', skipfooter=12, engine='python', thousands='.', decimal=',')
dados_atendimentos

## Fazendo slicing de dados.

É possível selecionar apenas uma parte do dataset para realizar algumas análises exploratórias. Por exemplo, podemos pegar dados de apenas duas colunas - agosto e setembro de 2008.

In [ ]:
dados_atendimentos[['2008/Ago', '2008/Set']].head()

Podemos realizar comparações com estes dados. Por exemplo, comparar gastos entre dois ou mais estados. Mas para isso, precisaremos coletar colunar que tenham dados de gastos em todos os meses pois, como vimos acima, existem colunas que não possuem dados. Uma forma de fazer esta tarefa é tirar a média do dataset.

In [ ]:
dados_atendimentos.mean()

Note que trouxe apenas dados a partir de 2007, mesmo tendo colunas que começam em '1992/Mar'. Isso porque a função média ignora colunas que não tem dados numéricos para fazer conta. Como estamos interessados nos nomes da colunas, podemos pegar todos eles, usando o índice.

In [ ]:
colunas_usaveis = dados_atendimentos.mean().index.tolist()
colunas_usaveis.insert(0, 'Unidade da Federação')
colunas_usaveis

Agora podemos fazer um novo sclicing dos nossos dados, pegando apenas colunas que contenham dados para comparação.

In [ ]:
dados_atendimentos[colunas_usaveis]

Quase tudo ok. O problema é que o índice do dataset está configurado com uma sequência numérica que vai do número 0 ao 26. Porém o ideal no nosso caso é que o índice seja o nome dos Estados. Podemos mudar isso.

In [ ]:
dados_atendimentos = dados_atendimentos[colunas_usaveis]
dados_atendimentos = dados_atendimentos.set_index('Unidade da Federação')
dados_atendimentos


Como o índice foi trocado, podemos buscar dados pelo mesmo. A instrução abaixo deve ser entendida como 'localize a linha (todos os meses e anos) cujo índice seja igual a 21 Maranhão'. 

In [ ]:
dados_atendimentos.loc['21 Maranhão']

## Plotando gráficos

Vamos plotar um gráfico do nosso dataset da forma mais simples possível.

In [ ]:
dados_atendimentos.plot()

Aconteceu um efeito indesejado. Ficou bem estranho! Ele entendeu que cada mês e ano são as linhas do gráfico, enquanto os Estados são as colunas. Por que isso aconteceu? Vamos observar o dataset novamente.

In [ ]:
dados_atendimentos.head()

O plot está pegando os dados das colunas e formando as linhas. O ideal é que os Estados fossem as colunas do nosso dataset, enquanto que os meses e anos fossem as linhas. Em outras palavras, precisamos rotacionar o nosso dataset. Isso é feito da seguinte forma:

In [ ]:
dados_atendimentos.T.head()

Isso torna-se possível usando o 'T'. Vamos plotar novamente.

In [ ]:
dados_atendimentos.T.plot()

O gráfico tem muitas linhas, porém passa a fazer mais sentido. A tarefa agora é 'fatiar' os dados para plotar somente aquilo que for de interesse na exploração de dados. Mesmo assim, podemos observar que na parte direita do gráfico os valores estão enormes. Isso é porque existe uma coluna totalizando os gastos dos Estados, prejudicando a exploração e legibilidade dos dados.

In [ ]:
dados_atendimentos.tail()

Precisamos remover esta coluna. Para não prejudicar o dataset original, vamos criar uma cópia dele sem a coluna total.

In [ ]:
dados_atendimentos_sem_total = dados_atendimentos.drop('Total', axis=1)
dados_atendimentos_sem_total

Agora sim podemos plotar novamente.

In [ ]:
dados_atendimentos_sem_total.T.plot()

O gráfico agora faz um pouco mais de sentido, mas ainda carece de ajustes.

## Desafios

### 1- Posicionar a legenda do gráfico de forma mais adequada

In [ ]:
dados_atendimentos_sem_total_plot = dados_atendimentos_sem_total.div(1000000)
plt.style.use('ggplot')
ax = dados_atendimentos_sem_total_plot.T.plot(figsize=(25,10))
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.2f}'))
plt.legend(loc='upper right', bbox_to_anchor=(1.23, 1.0), ncol=2, shadow=True, fancybox=True)
plt.ylabel('R$')
plt.title('Valor total gasto pelo SUS em atendimentos hospitalares por Estado (em milhão de reais)', fontsize=20)
plt.show()

### 2 - Plotar o gráfico de linha com apenas 5 estados de sua preferência

In [ ]:
plt.style.use('ggplot')
ax = dados_atendimentos_sem_total_plot.T.plot(figsize=(25,10), y=['31 Minas Gerais','42 Santa Catarina','33 Rio de Janeiro','35 São Paulo','28 Sergipe'])
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:,.2f}"))
plt.legend(loc="upper right", bbox_to_anchor=(1.18, 1.0), ncol=2, shadow=True, fancybox=True)
plt.ylabel("R$")
plt.title(label = "Valor total gasto pelo SUS em atendimentos hospitalares por Estado (em milhão de reais)", fontsize=20)
plt.show()

# Aula 3

## Outras técnicas de slicing de dados

Podemos pegar uma fatia dos dados por meio de algum critério de seleção. O primeiro que veremos é seleção por posição em um array. A instrução abaixo signifca pegar os seis primeiros dados, ou seja, de zero (primeiro elemento) até seis (último elemento). O número 7 foi colocado por se tratar de um 'exclusive', ou seja, 'pegue os dados de zero até seis, mas não o sete'.

In [ ]:
dados_atendimentos_sem_total[:7]

Uma outra forma possível é usando aleatoriedade. No exemplo abaixo, vamos solicitar aleatoriamente os dados de 8 estados.

In [ ]:
dados_atendimentos_sem_total.sample(n=8)

Se  rodarmos de novo a mesma instrução, veremos resultados diferentes.

In [ ]:
dados_atendimentos_sem_total.sample(n=8)

Caso seja desejável (por exemplo, em cenários onde a reprodutibilidade do estudo seja um requisito) podemos configurar uma semente para o gerados de números aleatórios.

In [ ]:
np.random.seed(547821)
dados_atendimentos_sem_total.sample(n=8)

Se rodarmos de novo com a mesma semente...

In [ ]:
np.random.seed(547821)
dados_atendimentos_sem_total.sample(n=8)

... Vamos obter o mesmo resultado. E podemos plotar o gráfico

In [ ]:
ax = dados_atendimentos_sem_total_plot.T.plot(figsize=(25,10))
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.2f}'))
plt.legend(loc="upper right", bbox_to_anchor=(1.23, 1.0), ncol=2, shadow=True, fancybox=True)
plt.ylabel("R$")
plt.title(label = "Valor total gasto pelo SUS em atendimentos hospitalares por Estado (em milhão de reais)", fontsize=20)
plt.show()

Existe um pico de valores no começo do gráfico que, a princípio, não sabemos o motivo. Além disso, não há clareza quanto às informações das linhas. Esta linha em vermelho em destaque se refere aos gastos do Estado de São Paulo ou Paraíba? Não tem como saber, pois o ggplot alocou a mesma cor para ambos os Estados! o_O

## Desafios

### 1 - Escolher uma palete de cores mais adequada do matplotlib

In [ ]:
colors = plt.cm.jet(np.linspace(0,1,27))
ax = dados_atendimentos_sem_total_plot.T.plot(figsize=(25,10))
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.2f}'))
plt.legend(loc="upper right", bbox_to_anchor=(1.23, 1.0), ncol=2, shadow=True, fancybox=True)
plt.ylabel("R$")
plt.title(label = "Valor total gasto pelo SUS em atendimentos hospitalares por Estado (em milhão de reais)", fontsize=20)
plt.show()

### 2 - Adicionar uma coluna mostrando a região

In [ ]:
def preenche_regioes(valor):
    switcher = {
        '1': 'Norte',
        '2': 'Nordeste',
        '3': 'Sudeste',
        '4': 'Sul',
        '5': 'Centro-Oeste'
    }
    return switcher.get(valor, 'Região do país inválida!')

for index, row in dados_atendimentos_sem_total_plot.iterrows():
    regiao = preenche_regioes(index[:1])
    dados_atendimentos_sem_total_plot.loc[index, 'Região']  = regiao

dados_atendimentos_sem_total_plot

### 3 - Formatar o gráfico de custos por mês dos 5 estados, deixando ele agradável 

Como esta tarefa já foi feita anteriormente, vamos reproduzir aqui, com alguns ajustes.

In [ ]:
colors = plt.cm.jet(np.linspace(0,1,5))
ax = dados_atendimentos_sem_total_plot.iloc[:, :-1].T.plot(figsize=(25,10), y=['31 Minas Gerais','42 Santa Catarina','33 Rio de Janeiro','35 São Paulo','28 Sergipe'])
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.2f}'))
plt.legend(loc="upper right", bbox_to_anchor=(1.1, 0.7), ncol=1, shadow=True, fancybox=True)
plt.ylabel("R$")
plt.title(label = "Valor total gasto pelo SUS em atendimentos hospitalares por Estado (em milhão de reais)", fontsize=20)
plt.show()

### 4 - Adicione o seu estado aos 5 estados plotados anteriormente

In [ ]:
colors = plt.cm.jet(np.linspace(0,1,6))
ax = dados_atendimentos_sem_total_plot.iloc[:, :-1].T.plot(figsize=(25,10), y=['31 Minas Gerais','42 Santa Catarina','33 Rio de Janeiro','35 São Paulo','28 Sergipe', '27 Alagoas'])
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.2f}'))
plt.legend(loc="upper right", bbox_to_anchor=(1.1, 0.7), ncol=1, shadow=True, fancybox=True)
plt.ylabel("R$")
plt.title(label = "Valor total gasto pelo SUS em atendimentos hospitalares por Estado (em milhão de reais)", fontsize=20)
plt.show()

Ainda há valores muito grandes no começo do gráfico, referentes ao mês de janeiro de 2008. Para este exercício, vamos considerar os dados de fevereiro de 2008 em diante.

In [ ]:
colors = plt.cm.jet(np.linspace(0,1,6))
colunas_de_interesse = dados_atendimentos_sem_total_plot.columns[6:]
dados_atendimentos_sem_total_plot = dados_atendimentos_sem_total_plot[colunas_de_interesse]
colors = plt.cm.jet(np.linspace(0,1,27))
ax = dados_atendimentos_sem_total_plot.iloc[:, :-1].T.plot(figsize=(25,10), y=['31 Minas Gerais','42 Santa Catarina','33 Rio de Janeiro','35 São Paulo','28 Sergipe', '27 Alagoas'])
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.2f}'))
plt.legend(loc="upper right", bbox_to_anchor=(1.1, 0.7), ncol=1, shadow=True, fancybox=True)
plt.ylabel("R$")
plt.title(label = "Valor total gasto pelo SUS em atendimentos hospitalares por Estado (em milhão de reais)", fontsize=20)
plt.show()

### 5 - Buscar os casos de dengue no Brasil e se os picos coincidem

No gráfico acima, podemos ver que existem alguns picos no Rio de Janeiro, sobretudo um perto de "2013/Fev". Soube-se por meio da imprensa que o Estado do Rio de Janeiro enfrentou um surto de dengue no referido ano. Para verificar se o surto coincide com os gastos hospitalares, recorri ao <a href='http://tabnet.datasus.gov.br/cgi/tabcgi.exe?sinannet/cnv/denguebr.def'>Tabnet Datasus</a> e pesquisei por notificações de casos prováveis de dengue por mês, filtrando por primeiros sintomas.

Inicialmente vamos importar os dados.

In [ ]:
dados_dengue_RJ = pd.read_csv('A001752189_28_143_208.csv', encoding='ISO-8859-1', skiprows=4, sep=';', skipfooter=17, thousands='.', decimal=',', engine='python')
dados_dengue_RJ= dados_dengue_RJ.set_index("Mês 1º Sintoma(s)")
dados_dengue_RJ

Vamos plotar um gráfico simples deste dataset.

In [ ]:
ax = dados_dengue_RJ.plot(figsize=(25,10))
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.2f}'))
plt.legend(loc="upper right", bbox_to_anchor=(1.1, 0.7), ncol=1, shadow=True, fancybox=True)
plt.ylabel("Número de notificações de casos prováveis")
plt.title(label = "Dengue em RJ - Notificação de casos prováveis por mês - Primeiros Sintomas - Ano de 2013", fontsize=20)
plt.show()

Ao plotar o gráfico dos estados, mas contendo apenas o Estado do Rio de Janeiro...

In [ ]:
gastos_hospitalares_RJ = dados_atendimentos_sem_total_plot.iloc[[18]]
gastos_hospitalares_RJ = gastos_hospitalares_RJ[["2012/Nov","2012/Dez","2013/Jan","2013/Fev","2013/Mar","2013/Abr","2013/Mai","2013/Jun","2013/Jul","2013/Ago","2013/Set","2013/Out","2013/Nov","2013/Dez"]]
ax = gastos_hospitalares_RJ.iloc[:, :-1].T.plot(figsize=(25,10))
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:,.2f}"))
plt.legend(loc="upper right", bbox_to_anchor=(1.23, 1.0), ncol=2, shadow=True, fancybox=True)
plt.ylabel("R$")
plt.title(label = "RJ - Gastos hospitalares por mês - em milhão de reais", fontsize=20)
plt.show()

... notamos que há uma coincidência entre o aumento de gastos hospitalares e a notificação de casos prováveis de dengue no período.

### 6 - Plotar o gráfico dos custos apenas dos estados da região sudeste e verificar se os picos de 2013/Fev teve comportamento similar em todos os demais estados da região

Primeiro, vamos criar um dataset a partir do nosso de estudo, porém contendo apenas os estados da região Sudeste.

In [ ]:
dados_atendimentos_sem_total_plot_sudeste = dados_atendimentos_sem_total_plot.loc[dados_atendimentos_sem_total_plot['Região'] == 'Sudeste']
dados_atendimentos_sem_total_plot_sudeste = dados_atendimentos_sem_total_plot_sudeste.iloc[: , :-1]
dados_atendimentos_sem_total_plot_sudeste

Agora vamos plotar um gráfico para o mesmo período da pesquisa no Datasus.

In [ ]:
colors = plt.cm.jet(np.linspace(0,1,4))
dados_atendimentos_sem_total_plot_sudeste = dados_atendimentos_sem_total_plot_sudeste[["2012/Nov","2012/Dez","2013/Jan","2013/Fev","2013/Mar","2013/Abr","2013/Mai","2013/Jun","2013/Jul","2013/Ago","2013/Set","2013/Out","2013/Nov","2013/Dez"]]
ax = dados_atendimentos_sem_total_plot_sudeste.T.plot(figsize=(25,10))
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:,.2f}"))
plt.legend(loc="upper right", bbox_to_anchor=(1.23, 1.0), ncol=2, shadow=True, fancybox=True)
plt.ylabel("R$")
plt.title(label = "Sudeste - Gastos hospitalares por mês - em milhão de reais", fontsize=20)
plt.show()

Vamos importar outro dataset, contendo agora os dados da região Sudeste.

In [ ]:
dados_dengue_sudeste = pd.read_csv('A004035189_28_143_208.csv', encoding='ISO-8859-1', skiprows=4, sep=';', skipfooter=17, thousands='.', decimal=',', engine='python')
dados_dengue_sudeste = dados_dengue_sudeste.set_index("Mês 1º Sintoma(s)")
dados_dengue_sudeste = dados_dengue_sudeste.iloc[1:] # A primeira linha do dataset vem em branco. Necessário remover a mesma
#dados_dengue_sudeste = dados_dengue_sudeste.set_index("Mês 1º Sintoma(s)")
dados_dengue_sudeste["ES"] = pd.to_numeric(dados_dengue_sudeste["ES"]) # Para reconhecer o valor das linhas como número, senão não plota o gráfico.
dados_dengue_sudeste["SP"] = pd.to_numeric(dados_dengue_sudeste["SP"]) # Para reconhecer o valor das linhas como número, senão não plota o gráfico.
dados_dengue_sudeste["RJ"] = pd.to_numeric(dados_dengue_sudeste["RJ"]) # Para reconhecer o valor das linhas como número, senão não plota o gráfico.
dados_dengue_sudeste

E agora vamos plotar o gráfico.

In [ ]:
colors = plt.cm.jet(np.linspace(0,1,4))
ax = dados_dengue_sudeste.iloc[: , :-1].plot(figsize=(25,10))
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:,.2f}"))
plt.legend(loc="upper right", bbox_to_anchor=(1.23, 1.0), ncol=2, shadow=True, fancybox=True)
plt.ylabel("Número de notificações de casos prováveis")
plt.title(label = "Dengue na região Sudeste - Notificação de casos prováveis por mês - Primeiros Sintomas - Ano de 2013", fontsize=20)
plt.show()

É possível verificar que as curvas de notificações de dengue estão bem próximas às curvas de gastos hospitalares para  região Sudeste.

### 7 - Adicionar seu estado escolhido novamente, deixe o gráfico informativo e tire conclusões sobre seus estados comparando com os demais

Vamos pegar os Estados do Sudeste mais Alagoas.

In [ ]:
dados_atendimentos_sem_total_plot_sudeste_alagoas = dados_atendimentos_sem_total_plot.iloc[[13,16,17,18,19]]
dados_atendimentos_sem_total_plot_sudeste_alagoas = dados_atendimentos_sem_total_plot_sudeste_alagoas.iloc[: , :-1]
dados_atendimentos_sem_total_plot_sudeste_alagoas

Vamos plotar o gráfico de gastos, agora contendo o Estado de Alagoas.

In [ ]:
colors = plt.cm.jet(np.linspace(0,1,5))
dados_atendimentos_sem_total_plot_sudeste_alagoas = dados_atendimentos_sem_total_plot_sudeste_alagoas[["2012/Nov","2012/Dez","2013/Jan","2013/Fev","2013/Mar","2013/Abr","2013/Mai","2013/Jun","2013/Jul","2013/Ago","2013/Set","2013/Out","2013/Nov","2013/Dez"]]
ax = dados_atendimentos_sem_total_plot_sudeste_alagoas.T.plot(figsize=(25,10))
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:,.2f}"))
plt.legend(loc="upper right", bbox_to_anchor=(1.23, 1.0), ncol=2, shadow=True, fancybox=True)
plt.ylabel("R$")
plt.title(label = "Sudeste e Alagoas- Gastos hospitalares por mês - em milhão de reais", fontsize=20)
plt.show()

Vamos recorrer novamente ao Datasus para incluir o Estado de Alagoas em um novo dataset.

In [ ]:
dados_dengue_sudeste_alagoas = pd.read_csv('A010611189_28_143_208.csv', encoding='ISO-8859-1', skiprows=4, sep=';', skipfooter=17, thousands='.', decimal=',', engine='python')
dados_dengue_sudeste_alagoas = dados_dengue_sudeste_alagoas.set_index("Mês 1º Sintoma(s)")
dados_dengue_sudeste_alagoas = dados_dengue_sudeste_alagoas.iloc[1:] # A primeira linha do dataset vem em branco. Necessário remover a mesma
#dados_dengue_sudeste = dados_dengue_sudeste.set_index("Mês 1º Sintoma(s)")
dados_dengue_sudeste_alagoas["ES"] = pd.to_numeric(dados_dengue_sudeste_alagoas["ES"]) # Para reconhecer o valor das linhas como número, senão não plota o gráfico.
dados_dengue_sudeste_alagoas["SP"] = pd.to_numeric(dados_dengue_sudeste_alagoas["SP"]) # Para reconhecer o valor das linhas como número, senão não plota o gráfico.
dados_dengue_sudeste_alagoas["RJ"] = pd.to_numeric(dados_dengue_sudeste_alagoas["RJ"]) # Para reconhecer o valor das linhas como número, senão não plota o gráfico.
dados_dengue_sudeste_alagoas["AL"] = pd.to_numeric(dados_dengue_sudeste_alagoas["AL"]) # Para reconhecer o valor das linhas como número, senão não plota o gráfico.
dados_dengue_sudeste_alagoas

Abaixo, plotaremos o gráfico.

In [ ]:
colors = plt.cm.jet(np.linspace(0,1,4))
ax = dados_dengue_sudeste_alagoas.iloc[: , :-1].plot(figsize=(25,10))
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:,.2f}"))
plt.legend(loc="upper right", bbox_to_anchor=(1.23, 1.0), ncol=2, shadow=True, fancybox=True)
plt.ylabel("Número de notificações de casos prováveis")
plt.title(label = "Dengue na região Sudeste e Alagoas- Notificação de casos prováveis por mês - Primeiros Sintomas - Ano de 2013", fontsize=20)
plt.show()

A curva de gastos do Estado de Alagoas manteve-se sem picos ao longo do ano de 2013, bem como a curva de notificações de casos prováveis de dengue. <a href='https://noticias.uol.com.br/saude/ultimas-noticias/redacao/2013/10/25/casos-de-dengue-quase-triplicam-em-2013.htm'>Notícias da época</a> informavam um quadro de triplicação de casos de dengue no país, com a região Sudeste concentrando 63,% do total de casos. A região Nordeste, onde fica Alagoas, era a terceira região com mais casos, totalizando 9,9% dos casos. Por estas notícias e pelos dados obtidos, faz sentido observar picos de gastos nos Estados da região Sudeste e certa estabilidade na região Nordeste, com destaque para Alagoas, o Estado comparado neste estudo.

# Aula 4

## Desafios

### 1 - Refaça as manipulações feitas em aulas e gaste um tempo analisando os dados e entendendo os efeitos das manipulações

Vamos pegar novamente o gráfico contendo gastos de seis estados.

In [ ]:
colors = plt.cm.jet(np.linspace(0,1,6))
colunas_de_interesse = dados_atendimentos_sem_total_plot.columns[6:]
dados_atendimentos_sem_total_plot_seis_estados = dados_atendimentos_sem_total_plot[colunas_de_interesse]
colors = plt.cm.jet(np.linspace(0,1,27))
ax = dados_atendimentos_sem_total_plot_seis_estados.iloc[:, :-1].T.plot(figsize=(25,10), y=['31 Minas Gerais','42 Santa Catarina','33 Rio de Janeiro','35 São Paulo','28 Sergipe', '27 Alagoas'])
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.2f}'))
plt.legend(loc="upper right", bbox_to_anchor=(1.1, 0.7), ncol=1, shadow=True, fancybox=True)
plt.ylabel("R$")
plt.title(label = "Valor total gasto pelo SUS em atendimentos hospitalares por Estado (em milhão de reais)", fontsize=20)
plt.show()

Existem dados desde o começo dos registros de dados. Vamos pegar conteúdo a partir de 2012, quando começou o pico de casos de dengue.

In [ ]:
colors = plt.cm.jet(np.linspace(0,1,6))
colunas_de_interesse = dados_atendimentos_sem_total_plot.columns[47:158] # dados a partir de Janeiro de 2012 até Março de 2021
dados_atendimentos_sem_total_plot_seis_estados = dados_atendimentos_sem_total_plot[colunas_de_interesse]
colors = plt.cm.jet(np.linspace(0,1,27))
ax = dados_atendimentos_sem_total_plot_seis_estados.iloc[:, :-1].T.plot(figsize=(25,10), y=['31 Minas Gerais','42 Santa Catarina','33 Rio de Janeiro','35 São Paulo','28 Sergipe', '27 Alagoas'])
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.2f}'))
plt.legend(loc="upper right", bbox_to_anchor=(1.1, 0.7), ncol=1, shadow=True, fancybox=True)
plt.ylabel("R$")
plt.title(label = "Valor total gasto pelo SUS em atendimentos hospitalares por Estado (em milhão de reais - desde Jan/2012 até Mar/2021)", fontsize=20)
plt.show()

Olhando o gráfico (olhando, não lendo), temos a impressão que o Estado de São Paulo gastou quase cinco vezes mais que o Estado de Santa Catarina (repare no espaço vertical entre as linhas). Porém, olhando o gráfico anterior temos a impressão que os gastos foram quase três vezes mais. O que está acontecendo?<br>
Na verdade, as janelas horizontal e vertical do gráfico estão modificadas de forma proposital a causar esta distorção. Podemos, por exemplo, plotar exatamente o mesmo gráfico anterior mas aumentando a janela do eixo Y para até 400 milhões.

In [ ]:
colors = plt.cm.jet(np.linspace(0,1,6))
colunas_de_interesse = dados_atendimentos_sem_total_plot.columns[47:158] # dados a partir de Janeiro de 2012 até Março de 2021
dados_atendimentos_sem_total_plot_seis_estados = dados_atendimentos_sem_total_plot[colunas_de_interesse]
colors = plt.cm.jet(np.linspace(0,1,27))
ax = dados_atendimentos_sem_total_plot_seis_estados.iloc[:, :-1].T.plot(figsize=(25,10), y=['31 Minas Gerais','42 Santa Catarina','33 Rio de Janeiro','35 São Paulo','28 Sergipe', '27 Alagoas'])
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.2f}'))
plt.legend(loc="upper right", bbox_to_anchor=(1.1, 0.7), ncol=1, shadow=True, fancybox=True)
plt.ylabel("R$")
plt.title(label = "Valor total gasto pelo SUS em atendimentos hospitalares por Estado (em milhão de reais - desde Jan/2012 até Mar/2021)", fontsize=20)
plt.ylim(0,400) # Aumentando a janela do eixc Y
plt.show()

Note como ficou mais parecido com o primeiro gráfico que plotamos nesta aula!<br>
E dá pra bagunçar mais ainda! E se começarmos os valores do eixo Y a partir de 50 milhões?

In [ ]:
colors = plt.cm.jet(np.linspace(0,1,6))
colunas_de_interesse = dados_atendimentos_sem_total_plot.columns[41:151] # dados a partir de Janeiro de 2012 até Março de 2021
dados_atendimentos_sem_total_plot_seis_estados = dados_atendimentos_sem_total_plot[colunas_de_interesse]
colors = plt.cm.jet(np.linspace(0,1,27))
ax = dados_atendimentos_sem_total_plot_seis_estados.iloc[:, :-1].T.plot(figsize=(25,10), y=['31 Minas Gerais','42 Santa Catarina','33 Rio de Janeiro','35 São Paulo','28 Sergipe', '27 Alagoas'])
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.2f}'))
plt.legend(loc="upper right", bbox_to_anchor=(1.1, 0.7), ncol=1, shadow=True, fancybox=True)
plt.ylabel("R$")
plt.title(label = "Valor total gasto pelo SUS em atendimentos hospitalares por Estado (em milhão de reais - desde Jan/2012 até Mar/2021)", fontsize=20)
plt.ylim(50,400) # Aumentando a janela do eixo Y
plt.show()

Este exercício nos mostra que é possível escolher os parâmetros do gráfico para contar uma mentira, mesmo usando dados que contém a verdade.

### 2 - Reflita sobre os problemas na manipulação de dados e discuta esse assunto no Discord

Estamos pesquisando e compartilhando artigos e textos que falam a respeito manipulação de gráficos.

# Aula 5

## Manipulação e interpretação de gráficos

Vamos reproduzir o mesmo gráfico de barras que fizemos na aula 1, porém com o dataframe atualizado e considerando o mês mais recente disponível, que é Março de 2021.

In [ ]:
mes_mais_recente = dados_atendimentos_sem_total.columns[-1]
mes_mais_recente
dados_mais_recentes = dados_atendimentos_sem_total.sort_values(by=mes_mais_recente, ascending=False)
dados_mais_recentes = dados_mais_recentes.div(1000000)
ax = dados_mais_recentes.plot(y='2021/Mar', kind='barh', figsize=(25,10), color='blue')
ax.invert_yaxis()
ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.2f}'))
plt.title(label = 'Valor total gasto pelo SUS em atendimentos hospitalares por Estado (em milhão de reais) - Mar/2021', fontsize=20)
plt.show()

## Desafios

### 1 - Buscar na documentação do Matplotlib como colocar um grid nos gráficos e adicionar nos gráficos de barra

O gráfico acima já está com grid, mas colocaremos a instrução explicitamente.

In [ ]:
ax = dados_mais_recentes.plot(y='2021/Mar', kind='barh', figsize=(25,10), color='blue')
ax.invert_yaxis()
ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.2f}'))
plt.title(label = 'Valor total gasto pelo SUS em atendimentos hospitalares por Estado (em milhão de reais) - Mar/2021', fontsize=20)
plt.grid(True, linestyle="-.")
plt.show()

### 2 - Fazer um gráfico e uma tabela do gasto dos outros estados em função do seu estado, ou qualquer outro de interesse

Primeiramente vamos fazer uma tabela em função do meu Estado, que é São Paulo.

In [ ]:
dados_mais_recentes_SP = dados_mais_recentes / dados_mais_recentes.loc["35 São Paulo"]
dados_mais_recentes_SP

E agora plotaremos o gráfico.

In [ ]:
idx = dados_mais_recentes_SP.index.get_loc('35 São Paulo')
cores = ['blue'] * dados_mais_recentes_SP.shape[0]
cores[idx] = 'red'
ax = dados_mais_recentes_SP.plot(y='2021/Mar', kind='barh', figsize=(25,10), color=cores)
ax.invert_yaxis()
ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.2f}'))
plt.title(label = 'Valor total gasto pelo SUS em atendimentos hospitalares por Estado (em milhão de reais) - Mar/2021', fontsize=20)
plt.grid(True, linestyle="-.")
plt.show()

Como o Estado de São Paulo é o que tem mais gastos, podemos usar outro estado para comparar. Por exemplo, Santa Catarina.

In [ ]:
dados_mais_recentes_SC = dados_mais_recentes / dados_mais_recentes.loc["42 Santa Catarina"]
dados_mais_recentes_SC

In [ ]:
idx = dados_mais_recentes_SC.index.get_loc('42 Santa Catarina')
cores = ['blue'] * dados_mais_recentes_SC.shape[0]
cores[idx] = 'red'
ax = dados_mais_recentes_SC.plot(y='2021/Mar', kind='barh', figsize=(25,10), color=cores)
ax.invert_yaxis()
ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.2f}'))
plt.title(label = 'Valor total gasto pelo SUS em atendimentos hospitalares por Estado (em milhão de reais) - Mar/2021', fontsize=20)
plt.grid(True, linestyle="-.")
plt.show()

### 3 - Fazer o cálculo proporcional a população do seu estado e mais um a sua escolha

Existe na Wikipedia uma tabela chamada <a href='https://pt.wikipedia.org/wiki/Lista_de_unidades_federativas_do_Brasil_por_popula%C3%A7%C3%A3o
wikipedia_url_populacao_por_estado = "https://pt.wikipedia.org/wiki/Lista_de_unidades_federativas_do_Brasil_por_popula%C3%A7%C3%A3o"
populacao = pd.read_html(wikipedia_url_populacao_por_estado, header=0, flavor="html5lib")[0]
populacao'>Lista de unidades federativas do Brasil por população</a>. Vamos usa-la para obter a população de cada Estado.<br>
Ela contém cinco colunas, das quais apenas "Unidade Federativa" e "População" servem para este desafio. Vamos recuperar as mesmas e transformar os dados da coluna 'População' para um tipo numérico.

In [ ]:
wikipedia_url_populacao_por_estado = "https://pt.wikipedia.org/wiki/Lista_de_unidades_federativas_do_Brasil_por_popula%C3%A7%C3%A3o"
populacao = pd.read_html(wikipedia_url_populacao_por_estado, header=0, flavor="html5lib")[0]
populacao = populacao[["Unidade federativa", "População"]]
populacao = populacao.set_index('Unidade federativa')
populacao['População'] = populacao['População'].str.replace(' ','')
populacao['População'] = pd.to_numeric(populacao['População'])
populacao

E agora vamos adicionar a população ao dataset.

In [ ]:
def get_populacao(estado):
    return populacao.loc[estado, 'População']

gastos_totais_por_populacao = dados_atendimentos_sem_total
gastos_totais_por_populacao['Total'] = gastos_totais_por_populacao.sum(axis=1)
gastos_totais_por_populacao = gastos_totais_por_populacao.sort_values('Total', ascending=False)

for index in gastos_totais_por_populacao.index:
    numero_habitantes = get_populacao(index[3:])
    gastos_totais_por_populacao.loc[index, 'População'] = numero_habitantes

gastos_totais_por_populacao['Gasto médio / habitante'] = gastos_totais_por_populacao['Total'] / gastos_totais_por_populacao['População']
gastos_totais_por_populacao = gastos_totais_por_populacao['Gasto médio / habitante']
gastos_totais_por_populacao = gastos_totais_por_populacao.sort_values(ascending=False)
gastos_totais_por_populacao

Com os dados em mãos, podemos plotar o gráfico.

In [ ]:
gastos_por_regiao = {
'1' : 'tab:red',
'2' : 'tab:blue',
'3' : 'tab:pink',
'4' : 'tab:green',
'5' : 'tab:orange'
}

cores = gastos_totais_por_populacao.index.str[0].map(gastos_por_regiao)
ax = gastos_totais_por_populacao.plot(y='Gasto médio / habitante', kind='bar', color=cores, figsize=(25,10), rot=75)
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('R$ {x:,.2f}'))
ax.set_ylabel('Gasto proporcional por população', fontweight='bold', fontsize='15')
ax.set_xticklabels(gastos_totais_por_populacao.index, size='large')
ax.set_xlabel('Estados', fontweight='bold', fontsize='18')
plt.title(label='Gastos proporcionais em atendimentos hospitalares por Estado - em R$', fontsize=20)
plt.text(21,-580,'Período: 2008 até 2020', verticalalignment='center', fontsize=13, fontweight='bold',color='black')
plt.text(0,-580,'Cores agrupadas por região',verticalalignment='center',fontsize=13,fontweight='bold',color='black')
plt.show()

### 4 - Faça uma análise dos dados analisados, levante hipóteses e compartilhe com a gente no Discord

Quando comparamos os gastos de saúde entre São Paulo e Santa Catarina, em valores absolutos, vemos que São Paulo gasta mais de cinco vezes. Entretanto, o gasto <i>per capita</i> do Estado do Sul do país é maior. <br> 
Historicamente o país tem se mostrado problemático quando o assunto é saúde pública. O Brasil, apesar de ter - por força da Constituição - um sistema público e universal de saúde, apresenta mais gastos privados na saúde que gastos públicos. <br>
Em 2014, por exemplo, <a href='https://www.scielosp.org/article/sdeb/2018.v42nspe2/37-47/pt/'>um estudo</a> mostrou que países da OCDE com sistema universal de saúde similar ao nosso, gastaram em média % do PIB nessa área, enquanto o Brasil gastou apenas 3,9%, considerando participação de Estados, Municípios e Governo Federal. <br>
Um ponto de grande agravamento foi a implementação de políticas de austeridade fiscal, como a <a href='https://www.in.gov.br/materia/-/asset_publisher/Kujrw0TZC2Mb/content/id/21295527/do1-2016-12-16-emenda-constitucional-n-95-21295459'>EF (Emenda Constitucional) 95/16</a>. A mesma estipulou um teto de gastos para o Governo, bem como congelamento de gastos com saúde em valores reais pelos próximos 20 anos. Com tal medida, o valor do PIB do Governo Federal destinado à saúde reduziria ao longo do tempo conforme a economia cresce. O impacto de médio e longo prazo é que neste mesmo período a população de idosos (mais vulneráveis e mais dependentes de serviços de saúde) <a href='http://www.analisepoliticaemsaude.org/oaps/boletim/edicao/8/'>dobrará</a>! <br>


### 5 - Reproduza as análises feitas em aulas, refletindo e levantando hipóteses

Para esta última atividade, tive a curiosidade de ver como está o gasto do Brasil com saúde em comparação a outros países.<br>
Foi usado neste comparativo a quantidade de dinheiro investido per capta anualmente. Usei <a href='https://data.oecd.org/healthres/health-spending.htm'>um database da OCDE</a> que contém estes valores, em dólares.<br>
A primeira etapa é importar os dados.

In [ ]:
dados_ocde = pd.read_csv('DP_LIVE_30052021041131100.csv', encoding='ISO-8859-1', engine='python')
dados_ocde

Como a coluna "LOCATION" está com caractéres indesejados, vamos renomar.

In [ ]:
dados_ocde.rename(columns={dados_ocde.columns[0]: 'LOCATION'}, inplace=True)
dados_ocde

Vamos restringir os dados ao critérios de interesse.<br>
Para estabelecer uma comparação minimamente equânime, adotei alguns critérios.<br>
O primeiro critério foi restringir os países. Este dataframe contém 52 países. Pesquisi países da OCDE que tem sistema universal de saúdes em moldes similares ao realizado no Brasil e preconizado pela nossa Constituição. <a href='https://www.poder360.com.br/midia/brasil-e-mesmo-unico-pais-com-amplo-sistema-publico-e-gratuito-de-saude/'>Segundo a pesquisa que realizei</a>, os países são Canadá, Espanha, Portugal, Reino Unido e Suécia. Então criamos um novo dataframe contendo estes países mais o Brasil.

In [ ]:
paises_sus = ['CAN', 'SWE', 'BRA', 'ESP','PRT', 'GBR']
dados_ocde = dados_ocde.loc[dados_ocde['LOCATION'].isin(paises_sus)]
dados_ocde

O próximo critério foi utilizar apenas dados de gastos compulsórios, ou seja, aquilo que efetivamente saiu dos cofres dos governos, uma vez que outros tipos de gastos registrados no dataset da OCDE são gastos voluntários e <i>out-of-pocket</i>.

In [ ]:
dados_ocde = dados_ocde.loc[dados_ocde['SUBJECT'] == 'COMPULSORY']
dados_ocde

A próxima etapa realizada foi restringir os anos entre 2000 e 2017. Embora os outros países da OCDE citados tenham dados desde os anos 1970, o Brasil tem dados a partir de 2000. Portando, vamos comparar todos os países no mesmo período.

In [ ]:
dados_ocde = dados_ocde.loc[dados_ocde['TIME'].between(2000,2017)]
dados_ocde

Por fim, filtramos o tipo de medição (MEASURE). O dataset mostra três formas de medir os gastos. A medida adotada aqui foi gastos como parcela do PIB de cada um dos países.

In [ ]:
dados_ocde = dados_ocde.loc[dados_ocde['MEASURE'] == 'PC_GDP']
dados_ocde

Agora vamos remover todas as demais colunas que não servirão para comparação.

In [ ]:
dados_ocde = dados_ocde.drop('SUBJECT', axis=1)
dados_ocde = dados_ocde.drop('INDICATOR', axis=1)
dados_ocde = dados_ocde.drop('MEASURE', axis=1)
dados_ocde = dados_ocde.drop('FREQUENCY', axis=1)
dados_ocde = dados_ocde.drop('Flag Codes', axis=1)
dados_ocde

Para facilitar a análise e a plotagem de dados, seria ideal que cada país fosse uma coluna no dataset, contendo os valores de gastos <i>per capta</i> em suas linhas, tendo os anos como índices. Existe uma função no Pandas chamada "pivot_table", que nos permite transformar o dataset para esta estrutura.

In [ ]:
dados_ocde = pd.pivot_table(dados_ocde, values='Value', index=['TIME'], columns=['LOCATION'])
dados_ocde

Para melhorar a legibilidade da tabela e do gráfico que faremos a seguir, vamos trocar os nomes das colunas para os nomes reais dos países.

In [ ]:
def preenche_paises(pais):
    switcher = {
        'CAN': 'Canadá',
        'ESP': 'Espanha',
        'BRA': 'Brasil',
        'SWE': 'Suécia',
        'PRT': 'Portugal',
        'GBR': 'Reino Unido'
    }
    return switcher.get(pais, 'País não encontrado!')



for col in dados_ocde.columns:
    dados_ocde.rename(columns={col: preenche_paises(col)}, inplace=True)
    
dados_ocde

Tudo pronto, podemos plotar o gráfico.

In [ ]:
colors = plt.cm.jet(np.linspace(0,1,6))
ax = dados_ocde.plot(figsize=(25,10))
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.2f} k'))
ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:.0f}'))
plt.legend(loc="upper right", bbox_to_anchor=(1.1, 0.7), ncol=1, shadow=True, fancybox=True)
plt.ylabel("US$")
plt.title(label = "Gastos anuais per capta de países da OCDE com saúde pública - Em milhares de dólares - Dados de 2000 até 2017", fontsize=20)
plt.show()

Como podemos visualizar, os gastos per capta em saúde do Brasil estão e historicamente sempre estiveram <b>MUITO ATRÁS</b> dos demais países.<br>
Entretando, é importante mencionar que entre os países mencionados, <a href='https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population'>nenhum deles chega nem perto de ter 210 milhões de habitantes</a> como o Brasil. Entre os países comparados, Reino Unido é o que tem mais, com 66 milhões de habitantes.<br>
Além disso, o PIB per capta do Brasil <a href='https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)_per_capita'>é o menor entre os países comparados</a>, deixando clara a fragilidade econômica (PIB Total) e a desigualdade econômica e social entre os brasileiros (PIB per capta). 